
# Lineare Regression mit einem Merkmal

Dieses Notebook zeigt Schritt für Schritt, wie wir mit scikit-learn eine einfache lineare Regression aufbauen. Der Fokus liegt darauf, die wichtigsten Konzepte zu verstehen und nicht auf möglichst ausgefeilten Tricks.



Wir gehen in vier kurzen Schritten vor:

1. Daten verstehen und visualisieren.
2. Ein Regressionsmodell trainieren.
3. Vorhersagen erzeugen und Kennzahlen berechnen.
4. Ergebnisse visualisieren und interpretieren.


In [1]:

import pandas as pd
import altair as alt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score



## 1. Daten kennenlernen

Wir verwenden `data/data-ads.csv`. Jede Zeile beschreibt, wie viel Budget (`ads`) für Werbung ausgegeben wurde und wie hoch die Verkäufe (`sales`) waren. So können wir untersuchen, ob sich höhere Ausgaben lohnen.


In [2]:

df = pd.read_csv("data/data-ads-2.csv")
df.head()


,sales,ads
0,8300,300
1,5900,550
2,4400,600
3,3100,650
4,5700,700



Ein erster Blick auf Basisstatistiken hilft zu verstehen, in welchen Größenordnungen sich die Variablen bewegen und ob Ausreißer auffallen.


In [4]:

df.describe()


,sales,ads
count,100.000000,100.000000
mean,37000.000000,7298.000000
std,20281.732837,4365.283843
min,3100.000000,300.000000
25%,19300.000000,3537.500000
50%,37150.000000,7350.000000
75%,52275.000000,10737.500000
max,76500.000000,14950.000000



Die Korrelation ist ein grober Indikator dafür, wie stark beide Variablen zusammenhängen.


In [5]:

df.corr(numeric_only=True).round(2)


,sales,ads
sales,1.00,0.99
ads,0.99,1.00



Ein Streudiagramm macht sichtbar, ob sich ein ungefähr linearer Trend erkennen lässt.


In [6]:

scatter = (
    alt.Chart(df)
    .mark_circle(size=80, color="#1f77b4")
    .encode(x=alt.X("ads", title="Werbebudget"), y=alt.Y("sales", title="Verkäufe"))
    .properties(title="Werbung vs. Verkäufe")
)
trend = scatter.transform_regression("ads", "sales").mark_line(color="#d62728", size=3)
scatter + trend


alt.LayerChart(...)


## 2. Modell aufbauen

Wir nutzen nur ein Merkmal (`ads`). Das Ziel ist `sales`. Die lineare Regression schätzt eine Gerade, die den Trend möglichst gut beschreibt.


In [7]:

feature = "ads"
target = "sales"

X = df[[feature]]
y = df[target]



Jetzt trainieren wir das Modell. `fit` passt Achsenabschnitt und Steigung so an, dass der Fehler minimiert wird.


In [8]:

model = LinearRegression()
model.fit(X, y)


,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False



Die Parameter des Modells lassen sich einfach interpretieren: die Steigung zeigt, wie stark `sales` pro zusätzlicher Einheit in `ads` steigt.


In [9]:

coef_table = pd.DataFrame(
    {
        "Parameter": ["Achsenabschnitt", "Steigung"],
        "Wert": [model.intercept_, model.coef_[0]],
    }
)
coef_table


,Parameter,Wert
0,Achsenabschnitt,3335.474366
1,Steigung,4.612843



## 3. Vorhersagen und Kennzahlen

Wir ergänzen den Datensatz um die Vorhersage und vergleichen sie mit den echten Verkäufen.


In [10]:

df = df.assign(prediction=model.predict(X))
df[["ads", "sales", "prediction"]].head()


,ads,sales,prediction
0,300,8300,4719.327160
1,550,5900,5872.537822
2,600,4400,6103.179954
3,650,3100,6333.822087
4,700,5700,6564.464219



Zur Bewertung nutzen wir die mittlere quadratische Abweichung (MSE), ihre Quadratwurzel (RMSE) und das Bestimmtheitsmaß $R^2$.


In [11]:

mse = mean_squared_error(y, df["prediction"])
rmse = mse ** 0.5
r2 = r2_score(y, df["prediction"])

metrics = pd.Series({"MSE": mse, "RMSE": rmse, "R^2": r2}, name="Modellgüte")
metrics.to_frame("Wert").round(3)


,Wert
MSE,5816637.864
RMSE,2411.771
R^2,0.986



Eine gemeinsame Grafik zeigt, wie dicht die Modellgerade an den Beobachtungen liegt.


In [12]:

df_sorted = df.sort_values(feature)
line = (
    alt.Chart(df_sorted)
    .mark_line(color="#d62728", size=3)
    .encode(x=alt.X("ads", title="Werbebudget"), y=alt.Y("prediction", title="Vorhersage"))
)
scatter + line


alt.LayerChart(...)


## 4. Merksatz

- Eine einfache lineare Regression reicht, wenn nur ein klarer Einflussfaktor existiert.
- Visualisierungen helfen, Annahmen (z. B. Linearität) zu prüfen.
- Metriken wie RMSE oder $R^2$ liefern eine Zahl, mit der Modelle verglichen werden können.
